In [1]:
# https://github.com/susanli2016/NLP-with-Python/blob/master/Doc2Vec%20Consumer%20Complaint_3.ipynb

import pyTigerGraph as tg 
from pyTigerGraph import gsql
from pyTigerGraph import graphToDataFrame

conn = tg.TigerGraphConnection(graphname="Yelp")
shell = gsql.Gsql(conn, certNeeded=False)

#conn.apiToken = conn.getToken(shell.createSecret())
tgDf = graphToDataFrame.graphToDataFrame(conn)

In [2]:
import pandas as pd 
import numpy as np 

from gensim.models import Doc2Vec
import gensim
from gensim.models.doc2vec import TaggedDocument

In [3]:
df = tgDf.getVertexDataframe("Review")
df.head()

,v_id,v_type,Stars,Useful,Funny,Cool,Text,DateOfReview,Testing,Features
0,VUPQUArwOPNQsQZ-eIAJGQ,Review,4,1,0,0,Really amazing service! Our garage spring brok...,2012-10-08 22:32:36,False,"[-0.17631, -0.10613, 0.62154, -0.20819, -0.106..."
1,dAvzp4bpMqwmP6WPK4zAzA,Review,3,0,0,0,at least the portions are big....the food is o...,2010-11-22 05:08:15,True,"[-0.24622, -0.07173, 0.56835, -0.19758, -0.112..."
2,Zvx1qa_VvdX-dZmUMj-6vQ,Review,4,0,0,0,Not sure if I saw anything related to a 'baker...,2011-04-22 00:02:27,False,"[-0.20724, -0.09201, 0.60906, -0.22314, -0.124..."
3,VX_t0CoEqswArOh65-j1PA,Review,3,3,0,0,I have to agree with most of the reviewers on ...,2010-05-15 16:39:01,False,"[-0.23609, -0.10467, 0.66664, -0.24914, -0.144..."
4,Pg7mjTzfQpKko4_mrGY39A,Review,5,3,1,1,Fast service the woman who did my hair was gre...,2011-04-25 04:55:20,False,"[-0.21369, -0.11698, 0.61554, -0.19465, -0.105..."


In [4]:
df.shape

(6911, 10)

In [5]:
cols = ['Stars', 'Useful', 'Funny', 'Cool', 'Text', 'DateOfReview']
df['combined'] = df[cols].apply(lambda row: " ".join(row.values.astype(str)), axis=1)

In [6]:
df.head()

,v_id,v_type,Stars,Useful,Funny,Cool,Text,DateOfReview,Testing,Features,combined
0,VUPQUArwOPNQsQZ-eIAJGQ,Review,4,1,0,0,Really amazing service! Our garage spring brok...,2012-10-08 22:32:36,False,"[-0.17631, -0.10613, 0.62154, -0.20819, -0.106...",4 1 0 0 Really amazing service! Our garage spr...
1,dAvzp4bpMqwmP6WPK4zAzA,Review,3,0,0,0,at least the portions are big....the food is o...,2010-11-22 05:08:15,True,"[-0.24622, -0.07173, 0.56835, -0.19758, -0.112...",3 0 0 0 at least the portions are big....the f...
2,Zvx1qa_VvdX-dZmUMj-6vQ,Review,4,0,0,0,Not sure if I saw anything related to a 'baker...,2011-04-22 00:02:27,False,"[-0.20724, -0.09201, 0.60906, -0.22314, -0.124...",4 0 0 0 Not sure if I saw anything related to ...
3,VX_t0CoEqswArOh65-j1PA,Review,3,3,0,0,I have to agree with most of the reviewers on ...,2010-05-15 16:39:01,False,"[-0.23609, -0.10467, 0.66664, -0.24914, -0.144...",3 3 0 0 I have to agree with most of the revie...
4,Pg7mjTzfQpKko4_mrGY39A,Review,5,3,1,1,Fast service the woman who did my hair was gre...,2011-04-25 04:55:20,False,"[-0.21369, -0.11698, 0.61554, -0.19465, -0.105...",5 3 1 1 Fast service the woman who did my hair...


In [7]:
from bs4 import BeautifulSoup
import re
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['combined'] = df['combined'].apply(cleanText)

In [8]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [9]:
taggedDf = df.apply(lambda r: TaggedDocument(words=tokenize_text(r['combined']), tags=[r.v_id]), axis=1)

In [10]:
taggedDf[20]

TaggedDocument(words=['thought', 'the', 'prices', 'were', 'awesome', 'and', 'the', 'food', 'the', 'food', 'actually', 'impressed', 'me', 'and', "'m", 'hard', 'to', 'impress', 'like', '2008-05-27', '08:27:53'], tags=['X0oUoChbouE-YtPLp16G-Q'])

In [11]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [12]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(taggedDf.values)])

100%|██████████| 6911/6911 [00:00<00:00, 2131384.92it/s]


In [13]:
%%time
from sklearn import utils

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(taggedDf.values)]), total_examples=len(taggedDf.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 6911/6911 [00:00<00:00, 852378.48it/s]
CPU times: user 1min 27s, sys: 14.8 s, total: 1min 42s
Wall time: 35.9 s


In [14]:
def upsertReviewVector(row):
    tokenized = tokenize_text(row["Text"])
    tagged = TaggedDocument(words=tokenized, tags=[row.v_id])
    vec = model_dbow.infer_vector(tagged.words, steps=20).tolist()
    conn.upsertVertex("Review", row["v_id"], {"Features": vec})

In [15]:
df.apply(lambda row: upsertReviewVector(row), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
6906    None
6907    None
6908    None
6909    None
6910    None
Length: 6911, dtype: object